In [17]:
import sys
sys.path.append('../..')
import numpy as np
import matplotlib.pyplot as plt

# PyTorch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.utils import make_grid
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import ds_ear_siamese
import transforms_data as td
from helpers import cuda_conv
import metrics as M
from siamese_network_train import Training
from ContrastiveLossFunction import ContrastiveLoss
from NN_Siamese import SiameseNetwork

from PIL import Image
import glob

In [18]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    MODEL_DIR = './models/model_MN_unfreezed1415161718.pt'
    RESIZE_SMALL = False
    DATABASE_FOLDER = './embeddings/'

In [19]:
model = torch.load(Config.MODEL_DIR)
model.to('cpu')
preprocess = td.transforms_siamese_verification( td.get_resize(Config.RESIZE_SMALL) )


In [20]:
def image_pipeline(input_, preprocess):
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.RESIZE_SMALL)[0], td.get_resize(Config.RESIZE_SMALL)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)

    return input_.type('torch.FloatTensor')
    
    # if cuda.is_available():
    #     return input_.type('torch.cuda.FloatTensor')
    # else:
    #     return input_.type('torch.FloatTensor')

In [21]:
loaded_embeddings = np.load(Config.DATABASE_FOLDER+'alexander_bec.npy', allow_pickle=True)

img = Image.open('/nfshome/lentzsch/Documents/Bachelorarbeit/auth_dataset/alexander_bec001.png')
new_embedding = model(Variable(image_pipeline(img,preprocess))).cpu()


In [22]:
for embedding in loaded_embeddings:
    dis = F.pairwise_distance(Variable(embedding),Variable(new_embedding))
    print(dis.item())

RuntimeError: The size of tensor a (27) must match the size of tensor b (1280) at non-singleton dimension 1